In [1]:
! pip install openai pandas instructor

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import os
import openai
import instructor
from pydantic import BaseModel, Field
from typing import List
from openai import OpenAI

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from IPython.display import HTML, display
display(HTML("<style>div.output_scroll { height: 300px; }</style>"))


client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])

instructor.patch(client)


In [3]:
from pydantic import BaseModel, Field, field_validator

class ConnectionGameGroup(BaseModel):
    name: str = Field(default="theme_name", description="the common theme name that connects the 4 words")
    group: List[str] = Field(default=["word1", "word2", "word3", "word4"],
                             description="A group of words with a common thread")
    @field_validator("group")
    def validate_group_length(cls, group):
        if len(group) != 4:
            raise ValueError("The group should contain exactly 4 words")
        return group

class ConnectionGame(BaseModel):
    easy: ConnectionGameGroup = Field(
        ...,
        alias="Group 1 (Easy)")
    medium: ConnectionGameGroup = Field(
        ...,
        alias="Group 2 (Medium)")
    hard: ConnectionGameGroup = Field(
        ...,
        alias="Group 3 (Hard)")
    very_hard: ConnectionGameGroup = Field(
        ...,
        alias="Group 4 (Very Hard)")

In [4]:

messages = [
    {"role": "system", "content":
        """You are a connection game generator. You can create a game with 16 words. You will structure your response with four groups, each containing four words. Label each group and ensure the words in each group are connected by a common thread. Each group can contain different types of words, i.e., a combination of nouns, verbs, and adjectives. Try use narrow and uncommon themes as the group names, and try to avoid using themes that are too board and common, such as 'animal', 'fruit', 'color'. Rank the groups from 'Easy' to 'Very Hard' based on how common the group name (theme) is.
        Here is one example:
        Group 1 (Easy): name='Primates', group=['baboon', 'bonobo', 'giboon', 'gorilla']
        Group 2 (Medium): name='religious figures' group=['cardinal', 'lama', 'monk', 'paster']
        Group 3 (Hard): name='chutney varieties', group=['mango', 'mint', 'tamarind', 'tomato']
        Group 4 (Very Hard): name='imitate', group=['ape', 'mime', 'mirror', parrot']
        """},
    {"role": "user", "content": "create a connection game for me"}
]


In [5]:
n_games = 5
for i in range(n_games):
    print(f"Connections game: {i+1}")
    connection_game: ConnectionGame = client.chat.completions.create(
        model="gpt-3.5-turbo-0613",
        response_model=ConnectionGame,
        messages=messages
    )

    print(connection_game.easy)
    print(connection_game.medium)
    print(connection_game.hard)
    print(connection_game.very_hard)
    print()


Connections game: 1
name='Famous Bridges' group=['Golden Gate', 'Brooklyn', 'Tower', 'Sydney Harbour']
name='Rare Gemstones' group=['Kashmir Sapphire', 'Musgravite', 'Serendibite', 'Grandidierite']
name='Ancient Civilizations' group=['Maya', 'Inca', 'Aztec', 'Mesopotamia']
name='Quantum Physics Concepts' group=['Entanglement', 'Superposition', 'Wave-particle duality', 'Quantum teleportation']

Connections game: 2
name='US Presidents' group=['Washington', 'Lincoln', 'Kennedy', 'Obama']
name='World Landmarks' group=['Eiffel Tower', 'Great Wall', 'Taj Mahal', 'Machu Picchu']
name='Programming Languages' group=['Python', 'Ruby', 'Java', 'C++']
name='Mathematical Constants' group=['Pi', "Euler's Number", 'Golden Ratio', 'Sqrt(2)']

Connections game: 3
name='Types of Dogs' group=['Labrador Retriever', 'Poodle', 'Bulldog', 'Chihuahua']
name='Planets in the Solar System' group=['Mars', 'Neptune', 'Saturn', 'Mercury']
name='Programming Languages' group=['Python', 'Ruby', 'Java', 'C++']
name='Ch